# Notes

**Hard Constraint:**  
- Each delivery locaiton must be visited exactly once
- Total **demand** of each vehicle route must not exceed its maximum **capacity**

**Self Cosntraint:**  
- Minimize cost required to meet all demands

**Assumption:**  
- Vehicles start and end their routes at the same depot location
- Each vehicle only travels one round trip (depart from depot and back to the depot)
- There is no limit on the number of vehicles.
- Travel times between any 2 locations are the same in both directions
- Deliveries can be made at any time, no time windows for deliveries
- Vehicle travel distance is calculated using Euclidian distance formula.

**Sample Output:**  
Vehicle 1 (Type A):  
Round Trip Distance: 41.861 km, Cost: RM 50.23, Demand: 22  
Depot -> C2 (9.072 km) -> C6 (1.834 km) -> C8 (19.343 km) -> Depot (11.612 km)  

Vehicle 2 (Type B):
...

# Implementation

In [1]:
import numpy as np
import pandas as pd

In [2]:
# id with 0 = depot
nodes = [
    { 'id': 0, 'latitude': 4.4184, 'longitude': 114.0932, 'demand': 0 },
    { 'id': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5 },
    { 'id': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8 },
    { 'id': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3 },
    { 'id': 4, 'latitude': 4.3184, 'longitude': 113.9932, 'demand': 6 },
    { 'id': 5, 'latitude': 4.4024, 'longitude': 113.9932, 'demand': 5 },
    { 'id': 6, 'latitude': 4.4142, 'longitude': 113.9896, 'demand': 8 },
    { 'id': 7, 'latitude': 4.4804, 'longitude': 114.0127, 'demand': 3 },
    { 'id': 8, 'latitude': 4.3818, 'longitude': 114.0734, 'demand': 6 },
    { 'id': 9, 'latitude': 4.4935, 'longitude': 114.2034, 'demand': 5 },
    { 'id': 10, 'latitude': 4.4932, 'longitude': 114.1322, 'demand': 8 },
]

vehicles = [
    { 'vehicle': 'A', 'capacity': 25, 'cost': 1.2 },
    { 'vehicle': 'B', 'capacity': 30, 'cost': 1.5 }
]

In [65]:
import math

class Node:
  def __init__(self, id, latitude, longitude, demand):
    self.id = id
    self.latitude = latitude
    self.longitude = longitude
    self.demand = demand
    
  def calculate_distance(self, node):
    return 100 * math.sqrt(
      math.pow(node.longitude - self.longitude, 2) +
      math.pow(node.latitude - self.latitude, 2)
    )
  
class Vehicle:
  def __init__(self, vehicle, capacity, cost):
    self.vehicle = vehicle
    self.capacity = capacity
    self.cost = cost

class Path:
  def __init__(self): 
    self.nodes = []
  
  def add_node(self, node):
    self.nodes.append(node)

  def calculate_total_distance(self):
    total_distance = 0
    for i in range(len(self.nodes) - 1):
      total_distance += self.nodes[i].calculate_distance(self.nodes[i+1])

    return total_distance


In [6]:
import math 
import numpy as np

def calculate_saving_heuristics(
    depot_node,
    source_node,
    dst_node
):
  G = 2
  F = 1

  intermediate_result = source_node.calculate_distance(depot_node) + depot_node.calculate_distance(dst_node)
  saving_heuristic = intermediate_result - (G * source_node.calculate_distance(dst_node)) + (F * abs(intermediate_result))

  return saving_heuristic

def calculate_load_influence(
    current_capacity,
    maximum_capacity,
    demand
):
  return (current_capacity + demand) / maximum_capacity

def calculate_probs(
    pheromone,
    depot_node,
    source_node,
    dst_node,
    current_capacity,
    maximum_capacity,
):
  ALPHA = 1
  BETA = 1
  GAMMA = 1
  DELTA = 1

  inverse_distance = (1 / source_node.calculate_distance(dst_node)) * BETA
  pheromone_influence = pheromone * ALPHA
  saving_heuristic = calculate_saving_heuristics(depot_node, source_node, dst_node) * GAMMA
  load_influence = calculate_load_influence(current_capacity, maximum_capacity, dst_node.demand) * DELTA

  return inverse_distance * pheromone_influence * saving_heuristic * load_influence

def calculate_next_node(
    vertices,
    pheromone_matrix,
    visited_nodes,
    current_node,
    current_capacity,
    maximum_capacity,
):
  # compute probs
  # decide next node
  probabilities_nodes = []
  
  for index, node in enumerate(vertices):
    if index == 0 or visited_nodes[index]:
      probabilities_nodes.append(0) # skip depot and visited nodes

    else:
      # calculate probability
      prob = calculate_probs(
        pheromone_matrix[current_node][index],
        vertices[0],
        vertices[current_node],
        node,
        current_capacity,
        maximum_capacity
      )
      
      probabilities_nodes.append(prob)

  # select next node
  probabilities_nodes = np.array(probabilities_nodes)
  probabilities_nodes = probabilities_nodes / probabilities_nodes.sum()
  next_node = np.argmax(probabilities_nodes) 
  
  return next_node # index of next node

def main(
    dataset,
    n_iter,
    n_ants,
    evaporation_rate,
    pheromone_coefficient,
):
  current_iter = 0
  vertices = [Node(**node) for node in dataset]
  vehicles = [Vehicle(**vehicle) for vehicle in vehicles]
  Vehicle = vehicles[0] # default vehicle
  pheromone_matrix = np.ones((len(vertices), len(vertices)))

  while current_iter < n_iter:

    for _ in range(n_ants):
      visited_nodes = [False] * len(vertices)
      visited_nodes[0] = True # depot is source node
      current_node = 1 # skip depot
      current_capacity = 0

      # initiate path
      path = Path()
      path.add_node(vertices[0]) # add depot

      while not all(visited_nodes):
        # if havent visit all nodes yet, then continue this iteration

        # decide next node
        next_node = calculate_next_node(
          vertices,
          pheromone_matrix,
          visited_nodes,
          current_node,
          current_capacity,
          Vehicle.capacity # maximum capacity
        )

        if current_capacity + vertices[next_node].demand > Vehicle.capacity:
          # return to depot if cannot proceed to next node due to capacity
          next_node = 0

        else:
          # visit the next node
          visited_nodes[next_node] = True

        current_node = next_node # update current position
        path.add_node(vertices[next_node])

      current_node = 0 # return to depot
    
    # save the best solution

    # evaporate pheromone trails 
    pheromone_matrix = pheromone_matrix * evaporation_rate

    # update pheromone trails
    """
      1. Need best solution find so far
      2. Distance of best solution in this generation 
    """

    current_iter += 1
    


In [5]:
pheromones = np.ones((10, 10))



array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [7]:
import math

def calculate_influence_load():
    pass 

def calculate_saving_heuristics(distance_matrix, i, j):
    G = 2.0
    F = 1

    intermediate_result = distance_matrix[i][0] + distance_matrix[0][j]
    result = intermediate_result - (G * distance_matrix[i][j]) + (F * math.abs(intermediate_result))

    return result

def calculate_probability(
    distance_matrix,
    pheromone,
    i,
    visited_nodes
):
    
    """
        prob / sum of prob of all unvisited nodes
        1. pheromone trail
        2. inverse of distance
        3. saving heuristics
        4. influence of load
    """
    probabilities_nodes = []
    for j, node in enumerate(visited_nodes):
        if j == 0 or node:
            # depot or visited node no need calculate
            continue 

        else:
            # calculate probability for unvisited nodes
            saving_heuristic = calculate_saving_heuristics(distance_matrix, i, j)
            inversed_distance  = 1 / distance_matrix[i][j] # reciprocal 
            prob_ij = pheromone[i][j] * inversed_distance * saving_heuristic * 1

    

def evaporate_pheromone():
    pass

def update_pheromone():
    pass

def create_distance_matrix(dataset):
    # adjacency matrix
    # initialize matrix of 0
    distance_matrix = np.zeros((len(dataset), len(dataset)))

    # calculate distance in kilometers for all edges
    for i in range(len(dataset)):
        for j in range(len(dataset)):
            distance_matrix[i][j] = euclidian_distance(dataset[i], dataset[j])

    return distance_matrix

def main(
    dataset,
    n_iter,
    n_ants, 
    alpha, 
    beta, 
    gamma, 
    delta
):
    distance_matrix = np.zeros((len(dataset), len(dataset)))
    pheromone_matrix = np.ones((len(dataset), len(dataset)))

    current_iter = 0
    while current_iter < n_iter:
        for ant in range(n_ants):
            current_node = 1 # skip depot
            visited_nodes = [False] * len(dataset)
            visited_nodes[0] = True # no need visit depot as it's source

            while not all(visited_nodes):
                # if haven't visit all nodes

                # calculate the probabilities of unvisited nodes, then decides which to go
                probabilities_nodes = []
                for index, node in enumerate(visited_nodes):
                    if index == 0 or node:
                        # depot or visited node no need calculate
                        continue 

                    else:
                        # calculate probability for unvisited nodes
                        prob_ij = calculate_probability(
                            distance_matrix,
                            pheromone_matrix,
                            current_node, # i
                            index # j
                        )

                        probabilities_nodes.append(prob_ij)

                









In [55]:
np.array([0, 1, 2, 3]) * (1/2)

array([0. , 0.5, 1. , 1.5])

array([0., 0., 0., 0., 0.])

In [53]:
items = ['Test1', 'Test2', 'Test3', 'Test4', 'Test5']

probs = [0.1, 0.1, 0.1, 0.5, 0.2]

items[np.argmax(probs)]

'Test4'

# References
1. https://www.researchgate.net/publication/228557657_Metaheuristics_for_Vehicle_Routing_Problems_with_Loading_Constraints_for_Wood_Products
2. https://github.com/afurculita/VehicleRoutingProblem/blob/master/src/ro/uaic/info/acs/VrpAcsSolver.java
3. https://github.com/pkonowrocki/CVRP_ACO